In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4

In [2]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

# import dotenv

# dotenv.load_dotenv()

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [ ]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [4]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [5]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
response["answer"]

'Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable. This process helps agents or models plan ahead and tackle difficult tasks effectively by dividing them into more achievable sub-tasks. Various techniques like Chain of Thought and Tree of Thoughts are used to decompose tasks into multiple steps for better understanding and execution.'

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

该链将输入查询的改写添加到我们的检索器中，以便检索包含对话的上下文。

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [6]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Common ways of extending the capabilities of LLMs through tool use include providing internet access for searches and information gathering, managing long-term memory effectively, utilizing GPT-3.5 powered agents for task delegation, and enabling file output for storing information. Additionally, performance evaluation techniques such as continuous self-review, constructive self-criticism, reflection on past decisions, and efficient task completion strategies can also enhance the overall performance of LLM-centered agents. Lastly, task decomposition methods involving LLM prompting, task-specific instructions, and human inputs can help in breaking down complex tasks into manageable steps for the LLM to process effectively.


下面我们实现第二种方案的简单示例，聊天记录存储在一个简单的字典中。LangChain 管理与Redis和其他技术的内存集成，以提供更强大的持久性。

实例RunnableWithMessageHistory为您管理聊天历史记录。它们接受带有键的配置（"session_id"默认情况下），该键指定要获取并添加到输入中的对话历史记录，并将输出附加到相同的对话历史记录。以下是一个例子：

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [8]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]


'Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This process helps agents, like autonomous systems, plan and execute tasks more efficiently. It can be achieved by prompting models with specific instructions, such as guiding them to think step by step or by using human inputs.'

In [11]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Common ways of extending the capabilities of language model agents include providing internet access for searches and information gathering, managing long-term memory effectively, utilizing GPT-3.5 powered agents for delegating simple tasks, and enabling file output for storing information. Performance evaluation involves continuously reviewing actions, self-critiquing behavior, reflecting on past decisions, and optimizing task completion efficiency. Task decomposition can be achieved through simple prompting by the language model, task-specific instructions, or human inputs to enhance problem-solving and decision-making processes.'

In [12]:
print(f'store:{store}')

store:{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content='Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This process helps agents, like autonomous systems, plan and execute tasks more efficiently. It can be achieved by prompting models with specific instructions, such as guiding them to think step by step or by using human inputs.'), HumanMessage(content='What are common ways of doing it?'), AIMessage(content='Common ways of extending the capabilities of language model agents include providing internet access for searches and information gathering, managing long-term memory effectively, utilizing GPT-3.5 powered agents for delegating simple tasks, and enabling file output for storing information. Performance evaluation involves continuously reviewing actions, self-critiquing behavior, reflecting on past decisions, and optimizing task completion efficiency. Task decom

In [13]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This process helps agents, like autonomous systems, plan and execute tasks more efficiently. It can be achieved by prompting models with specific instructions, such as guiding them to think step by step or by using human inputs.

User: What are common ways of doing it?

AI: Common ways of extending the capabilities of language model agents include providing internet access for searches and information gathering, managing long-term memory effectively, utilizing GPT-3.5 powered agents for delegating simple tasks, and enabling file output for storing information. Performance evaluation involves continuously reviewing actions, self-critiquing behavior, reflecting on past decisions, and optimizing task completion efficiency. Task decomposition can be achieved through simple prompting by the language model, task-specific instructions, or human inputs to en

https://python.langchain.com/v0.2/assets/images/conversational_retrieval_chain-5c7a96abe29e582bc575a0a0d63f86b0.png